In [1]:
import pandas as pd
import numpy as np
import rasterio
from rasterio import *
from rasterio.plot import show
from pyspatialml import Raster

from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE


In [2]:
predictors = pd.read_csv("out_data/forest_data_poland_lucas.csv", sep=" ",  index_col=False, na_values = 48,
                         header = None)
pd.set_option('display.max_columns',None)


Better variable names

In [3]:
predictors_names = pd.read_csv("out_data/available_features_names.csv", sep=",",  index_col=False)
predictors.columns = predictors_names.name.tolist()
predictors.tail(5)

,GLAD_mean100m,GLAD_median100m,GLAD_mode100m,GLAD_q1,GLAD_q3,CLC10m_mean,CLC10m_median,CLC10m_mode.,CLC10m_q1,CLC10m_q3,CLC_01.vrt,C-GLOPS_mean,C-GLOPS_median,C-GLOPS_mode,C-GLOPS_q1,C-GLOPS_q3,JAXA_q3,JAXA_q1,JAXA_mode,JAXA_median,JAXA_mean
23079,0.246136,0,0,0,1,0.60,1,1,0,1,0,0.191063,0.07,0.07,0.04,0.08,0.5,0.0,0.0,0.0,0.277736
23080,0.000000,0,0,0,0,0.00,0,0,0,0,0,0.016844,0.01,0.01,0.01,0.03,0.0,0.0,0.0,0.0,0.000000
23081,0.932173,1,1,1,1,0.95,1,1,1,1,1,0.402705,0.44,0.39,0.39,0.62,1.0,1.0,1.0,1.0,0.961699
23082,0.000000,0,0,0,0,0.00,0,0,0,0,0,0.043004,0.03,0.03,0.02,0.03,0.0,0.0,0.0,0.0,0.023998
23083,0.000000,0,0,0,0,0.09,0,0,0,0,0,0.098559,0.09,0.06,0.06,0.11,0.0,0.0,0.0,0.0,0.000000


Now the left hand side

In [4]:
outcome = pd.read_csv("out_data/LUCAS/LUCAS_locations.csv", sep=",",  index_col=False)
outcome.tail(5)

,X,Y,LC1,LC1_PERC,forest
23079,5.278000e+06,3.114000e+06,C22,100,1
23080,5.278000e+06,3.136000e+06,B11,100,0
23081,5.294000e+06,3.150000e+06,C10,100,1
23082,5.294000e+06,3.172000e+06,B11,92,0
23083,5.286000e+06,3.116000e+06,Bx1,98,0


In [5]:
print (len(outcome))
print (len(predictors))

23084
23084


In [6]:
outcome.head()

,X,Y,LC1,LC1_PERC,forest
0,5.214000e+06,3.064000e+06,B16,95,0
1,5.214000e+06,3.116000e+06,E20,100,0
2,5.214000e+06,3.388000e+06,C10,100,1
3,5.214000e+06,3.474000e+06,B16,1,0
4,4.792000e+06,3.082000e+06,B14,50,0


Split the Dataset

In [7]:
X    = predictors
Y    = outcome.iloc[:,4].to_frame()
feat = predictors.columns.values
print(X.shape)
print(Y.shape)
print(type(X))
print(type(Y))

(23084, 21)
(23084, 1)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


There are a handful of NAs

In [8]:
na_rows = X.apply(lambda x: np.any(pd.isna(x)), axis=1)
result = np.where(na_rows)[0]
print('number NA: ' + str(len(result)))

number NA: 1265


In [9]:
X = X.drop(X.index[result])
Y = Y.drop(Y.index[result])


In [10]:
print(X.shape)
print(Y.shape)

(21819, 21)
(21819, 1)


Create 4 dataset for training and testing the algorithm 

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=2023)
y_train = np.ravel(Y_train)
y_test = np.ravel(Y_test)

In [12]:
print(np.mean(y_train==1))
print(np.mean(y_test==1))

0.3208360069667247
0.3242896425297892


rfReg = RandomForestClassifier(n_estimators=2799,
                        max_features=3,
                        max_depth=250,
                        max_samples=0.3335,
                        n_jobs=-1,
                        oob_score = True)

n_features = 15  # specify the number of desired features

rfe = RFE(estimator=rfReg, n_features_to_select=n_features)


rfe.fit(X_train, y_train)

selected_features = predictors.columns[rfe.get_support()]

type(selected_features)

rfe.score(X_test, y_test)

predictors.columns[rfe.get_support()]

Use wandb to seach the model space 

In [13]:
import wandb

In [15]:
my_entity="hofer1991"
my_project="RF_Foresthybrid_table"

wandb.init(project=my_project)

wandb: Currently logged in as: hofer1991 (nodes). Use `wandb login --relogin` to force relogin


In [16]:
#wandb.sklearn.plot_confusion_matrix(y_train, dic_pred['train'])


In [17]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'BIC/val',
      'goal': 'maximize'   
    },
    'parameters': {
        'sel__n_features': {
            'distribution': 'int_uniform',
            'min' : 3 ,
            'max' : 10
        },
        'rf__max_features': {
            'distribution': 'int_uniform',
            'min' : 3 ,
            'max' : 10
        },
        'rf__max_samples':{
            'distribution': 'uniform',
            'min' : 0.3 ,
            'max' : 0.7
        },
        'rf__n_estimators':{
            'distribution': 'int_uniform',
            'min' : 500 ,
            'max' : 2000
        },
        'rf__max_depth':{
            'distribution': 'int_uniform',
            'min' : 50 ,
            'max' : 300
        }
    }
}

In [21]:
# Initialize a new sweep
# Arguments:
#     – sweep_config: the sweep config dictionary defined above
#     – entity: Set the username for the sweep
#     – project: Set the project name for the sweep
sweep_id = wandb.sweep(sweep_config, entity=my_entity, project=my_project)

Create sweep with ID: uq2lt86p
Sweep URL: https://wandb.ai/hofer1991/RF_Foresthybrid_table/sweeps/uq2lt86p


In [23]:
# Training and Evaluation routines for Sweeping
def performance_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc = (tp + tn) / (tn + tp + fp + fn)
#    tpr = tp / (tp + fn)
#    tnr = tn / (tn + fn)
#    bacc= (tpr + tnr) * 0.5
#    f1 = (tpr * tnr) / (tpr + tnr)
    return(acc)

def calculate_bic(y_true, y_pred, num_params):
    resid = y_true - y_pred
    sse = np.sum(resid**2)
    n = len(y_true)
    bic = num_params * np.log(n) + n * np.log(sse/n)
    return -bic

def evaluate_acc(model, X, y):
    y_pred = model.predict(X) 
    metric  = performance_metrics(y, y_pred)
    return(metric)
    

def evaluate_bic(model, X, y, num_params):
    y_pred = model.predict(X) 
    metric  = calculate_bic(y, y_pred, num_params )
    return(metric)
    
def train(config=None):
#    best_acc = -1
    with wandb.init(config=config):
        
        # Config is a variable that holds and saves hyperparameters and inputs
        config = wandb.config
        
        n_features = config.sel__n_features  # specify the number of desired features
        
#         n_features must be smaller or equal to rf__max_features
        
    
    
        rfReg = RandomForestClassifier(n_estimators=config.rf__n_estimators,
                              max_features=min(config.rf__max_features, n_features),
                              max_depth=config.rf__max_depth,
                              max_samples=config.rf__max_samples,
                              n_jobs=-1,
                              oob_score = True) 
        
      
    


        selector = RFE(rfReg, n_features_to_select=n_features)

        selector.fit(X_train, y_train)
  
        selected_features = predictors.columns[selector.get_support()]
        selected_features_log = list(predictors.columns[selector.get_support()])
        
        bic_train = evaluate_bic(selector,X_train, y_train, n_features)
        bic_test  = evaluate_bic(selector, X_test, y_test, n_features)
        
        acc_train = evaluate_acc(selector,X_train, y_train)
        acc_test  = evaluate_acc(selector, X_test, y_test)

        
        wandb.log({
               "BIC/train": bic_train,
               "BIC/val": bic_test,
                "Accuracy/train": acc_train,
                "Accuracy/val": acc_test, 
                "SelectedFeatures": selected_features_log
        })
        
    
        
        # Save the model with the best accuracy
        if bic_test < wandb.run.summary.get("best_accuracy", 0):
            print('bic_test: ',bic_test)
            wandb.run.summary["best_accuracy"] = bic_test
            model_name = "models/best_RF_Foresthybrid" + str(n_features) + "_featuresel.pkl"
            joblib.dump(selector, model_name)
    
        

In [24]:
wandb.agent(sweep_id, train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: yw06bai5 with config:
wandb: 	rf__max_depth: 151
wandb: 	rf__max_features: 3
wandb: 	rf__max_samples: 0.3710091095623782
wandb: 	rf__n_estimators: 566
wandb: 	sel__n_features: 8


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


Accuracy/train,▁
Accuracy/val,▁
BIC/train,▁
BIC/val,▁
Accuracy/train,0.95481
Accuracy/val,0.90009
BIC/train,33708.98599
BIC/val,25056.82847


wandb: Agent Starting Run: egruz733 with config:
wandb: 	rf__max_depth: 150
wandb: 	rf__max_features: 9
wandb: 	rf__max_samples: 0.3288761638549175
wandb: 	rf__n_estimators: 1992
wandb: 	sel__n_features: 4
wandb: Currently logged in as: hofer1991. Use `wandb login --relogin` to force relogin


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


...

Accuracy/train,▁
Accuracy/val,▁
BIC/train,▁
BIC/val,▁
Accuracy/train,0.94757
Accuracy/val,0.90119
BIC/train,32068.98863
BIC/val,25159.00974


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 6z1q56nz with config:
wandb: 	rf__max_depth: 285
wandb: 	rf__max_features: 8
wandb: 	rf__max_samples: 0.3208635556432811
wandb: 	rf__n_estimators: 1893
wandb: 	sel__n_features: 10


wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x7fe0aa7939a0> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe